# Altair heatmap and long-form data transformation

Altair is an information visualization library that is relatively easy to understand with clean and straightforward code. The caveat is that the original data needed to be manipulated to the appropriate form to be suitable with Altair. In this tutorial, I demonstrate the transformation from survey data to a long-form dataframe, then use this dataframe to create a complete heatmap with considerations of aesthetic features.

# Install and Setup Altair

If you have never used Altair before, chances are it is not already installed in your working environment. To install Altair, you can either execute the following command in pip:

```pip install altair```

Or using conda:

```conda install -c conda-forge altair```

I am not going to use one of the existing datasets in the example [vega_datasets](https://github.com/altair-viz/vega_datasets) in this guide, but if you do, you can install Altair along with these datasets using the following command in pip:

```pip install altair vega_datasets```


Or if you are using the conda manager, you do:

```conda install -c conda-forge altair vega_datasets```

Now, to make sure your visualizations are rendered correctly in Jupyter Notebook, you want to run the following code:

```alt.renderers.enable('default')```

To improve performance when working with large datasets, use the ```json``` data transformer by running the following code:

```alt.data_transformers.enable('json')```

# Demonstration

Now that you already have Altair installed, let's begin creating some awesome visualizations.

## I. Import Altair Library and Dataset

In [1]:
# import Altair
import altair as alt

alt.renderers.enable('default')

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

I will be using the [Food World Cup](https://github.com/fivethirtyeight/data/tree/master/food-world-cup) dataset by FiveThirtyEight. This is the dataset associated with two of their articles : [The FiveThirtyEight International Food Association’s 2014 World Cup](https://fivethirtyeight.com/features/the-fivethirtyeight-international-food-associations-2014-world-cup/) and [What is Americans’ Favorite Global Cuisine?](https://fivethirtyeight.com/features/what-is-americans-favorite-global-cuisine/)

In [2]:
# import Pandas for data manipulation

import pandas as pd

In [3]:
# read the original dataset
df = pd.read_csv('food-world-cup-data.csv')

In [4]:
# take a look at the head of the dataset
df.head()

,RespondentID,"Generally speaking, how would you rate your level of knowledge�of cuisines from different parts of the world?","How much, if at all, are you interested in cuisines from different parts of the world?",Please rate how much you like the traditional cuisine of Algeria:,Please rate how much you like the traditional cuisine of Argentina.,Please rate how much you like the traditional cuisine of�Australia.,Please rate how much you like the traditional cuisine of Belgium.,Please rate how much you like the traditional cuisine of Bosnia and Herzegovina.,Please rate how much you like the traditional cuisine of Brazil.,Please rate how much you like the traditional cuisine of Cameroon.,...,Please rate how much you like the traditional cuisine of Turkey.,Please rate how much you like the traditional cuisine of Cuba.,Please rate how much you like the traditional cuisine of Ethiopia.,Please rate how much you like the traditional cuisine of Vietnam.,Please rate how much you like the traditional cuisine of Ireland.,Gender,Age,Household Income,Education,Location (Census Region)
0,3308895255,Intermediate,Some,NaN,3.0,5.0,4.0,NaN,3.0,NaN,...,NaN,NaN,3.0,NaN,4.0,Male,18-29,"$100,000 - $149,999",Less than high school degree,West South Central
1,3308891308,Novice,Some,NaN,NaN,3.0,3.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,4.0,Male,18-29,"$100,000 - $149,999",Some college or Associate degree,West South Central
2,3308891135,Intermediate,A lot,3.0,4.0,NaN,3.0,NaN,4.0,5.0,...,NaN,4.0,5.0,4.0,3.0,Male,30-44,"$50,000 - $99,999",Graduate degree,Pacific
3,3308879091,Novice,Not much�,NaN,3.0,NaN,3.0,NaN,NaN,3.0,...,3.0,4.0,NaN,4.0,3.0,Male,45-60,"$0 - $24,999",Less than high school degree,New England
4,3308871671,Novice,Not much�,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,3.0,NaN,Male,30-44,"$25,000 - $49,999",High school degree,Pacific


## II. Manipulate Data to Long Form for Visualization

In [5]:
# remove long question in column names
df.columns = df.columns.str.replace('Please rate how much you like the traditional cuisine of', '')

In [6]:
# remove special characters in column names
df.columns = df.columns.str.replace('[:,.,�]', '')

<ipython-input-6-63a1872de865>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[:,.,�]', '')


In [7]:
# remove white spaces in column names
df.columns = df.columns.str.strip()

In [8]:
# rename some of the columns for accessibility
df = df.rename(columns={'RespondentID': 'ID',
                        'Generally speaking how would you rate your level of knowledgeof cuisines from different parts of the world?': 'cuisine_knowledge',
                        'How much if at all are you interested in cuisines from different parts of the world?': 'how_interested',
                        'Location (Census Region)': 'location',
                        'the Netherlands': 'The Netherlands'})

In [9]:
# see the columns that we got
df.columns

Index(['ID', 'cuisine_knowledge', 'how_interested', 'Algeria', 'Argentina',
       'Australia', 'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Cameroon',
       'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Ecuador', 'England',
       'France', 'Germany', 'Ghana', 'Greece', 'Honduras', 'Iran', 'Italy',
       'Ivory Coast', 'Japan', 'Mexico', 'The Netherlands', 'Nigeria',
       'Portugal', 'Russia', 'South Korea', 'Spain', 'Switzerland',
       'United States', 'Uruguay', 'China', 'India', 'Thailand', 'Turkey',
       'Cuba', 'Ethiopia', 'Vietnam', 'Ireland', 'Gender', 'Age',
       'Household Income', 'Education', 'location'],
      dtype='object')

### 1. First attempt

Let's say I want to create a visualization that shows how popular a cuisine is throughout different regions in the U.S. I can do so by creating a heatmap with country names on one axis and locations on the other. Since we have many more country names than we have locations, for legibility, we will want country names on the y-axis and location on the x-axis.

In [10]:
# first I specify all the country names we got
countries = ['Algeria', 'Argentina',
       'Australia', 'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Cameroon',
       'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Ecuador', 'England',
       'France', 'Germany', 'Ghana', 'Greece', 'Honduras', 'Iran', 'Italy',
       'Ivory Coast', 'Japan', 'Mexico', 'The Netherlands', 'Nigeria',
       'Portugal', 'Russia', 'South Korea', 'Spain', 'Switzerland',
       'United States', 'Uruguay', 'China', 'India', 'Thailand', 'Turkey',
       'Cuba', 'Ethiopia', 'Vietnam', 'Ireland']

In [11]:
# then I specify all the locations we got
locations = ['East North Central', 'East South Central', 'Middle Atlantic',
       'Mountain', 'New England', 'Pacific', 'South Atlantic',
       'West North Central', 'West South Central']

In [12]:
# create a new dataframe displaying the median score of each cuisine by locations
heatmap_df = df.groupby(['location'])[countries].median().reset_index()
heatmap_df

,location,Algeria,Argentina,Australia,Belgium,Bosnia and Herzegovina,Brazil,Cameroon,Chile,Colombia,...,United States,Uruguay,China,India,Thailand,Turkey,Cuba,Ethiopia,Vietnam,Ireland
0,East North Central,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0
1,East South Central,3.0,4.0,3.0,3.0,3.5,3.0,3.5,3.0,3.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,Middle Atlantic,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,4.0,3.0,4.0,3.0,3.5,3.0,3.0,3.0
3,Mountain,3.0,4.0,3.0,3.0,3.0,4.0,3.0,4.0,3.5,...,4.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0
4,New England,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,4.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0
5,Pacific,3.0,4.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,...,4.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0
6,South Atlantic,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,...,4.0,3.0,4.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0
7,West North Central,3.0,3.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,...,4.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0
8,West South Central,3.0,3.0,3.0,3.0,3.0,4.0,3.0,4.0,3.0,...,4.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0


My intitial approach was to produce a heatmap that shows how popular a cuisine is by geographical locations. However, after producing the above ```heatmap_df``` DataFrame, I came across a small problem. I realized that the potential graph would contain very little useful information and would not be effective of what I am trying to show because the number of respondents differ among different cuisine. For example, I assume that more people are familar with Italian cuisine versus Algerian cuisine in most locations. Therefore, the median result of each cuisine does not represent how popular the cuisine is. I decided to go with a different approach by showing how popular a cuisine is across **all** locations and how often the repondents rate them on the scale 1 to 5.

### 2. Second attempt

I proceed with this approach by first calculate the total number of repsondents who rated a particular cuisine among all who surveyed. These numbers are expected to be very different because some cuisine (e.g. Italian cuisine) are much more popular than others.

In [13]:
rate_total = []  # create an empty list contains total number of repondents for each country

for country in countries:    # for each country in the country list specifed above
    count = df.dropna(subset=[country])      # drop NaN from each of the country columns
    total = len(count)                       # count the number of entries excluding NaN responses
    rate_total.append(total)                 # append these number to the list

In [14]:
rate_tuples = list(zip(countries, rate_total))  # create tuples by zipping country names and its corresponding value
rate_total_df = pd.DataFrame(rate_tuples, columns = ['Country', 'Total'])  # make these tuples a dataframe
rate_total_df  # here is our dataframe

,Country,Total
0,Algeria,188
1,Argentina,470
2,Australia,483
3,Belgium,474
4,Bosnia and Herzegovina,156
5,Brazil,547
6,Cameroon,124
7,Chile,420
8,Colombia,383
9,Costa Rica,418


Now that I have the dataframe with total number of respondents who rated a particular cuisine, I use information on this dataframe to produce another dataframe with the percentage of respondents who rated a cuisine by the scale of 1 to 5.

In [15]:
rate_percentage = [] # create an empty list contains the percentage

for country in countries:   # for each country in the country list specifed above
    
    # count each value (1 to 5) of each country column     # divide by the total of respodents in rate_total_df
    # round this number to 2 decimals for readability in our heatmap
    percentage = round(df[country].value_counts() / rate_total_df.loc[rate_total_df['Country'] == country]['Total'].unique(), 2)
    # when each percentage is caculated, append the value to the list
    rate_percentage.append(percentage)

# create a new dataframe with the list of percentage, reset index
rate_percentage = pd.DataFrame(rate_percentage).reset_index()
# take a look at what we just produced
rate_percentage.head()

,index,1.0,2.0,3.0,4.0,5.0
0,Algeria,0.04,0.21,0.45,0.24,0.06
1,Argentina,0.01,0.04,0.48,0.38,0.10
2,Australia,0.02,0.07,0.56,0.28,0.07
3,Belgium,0.01,0.03,0.48,0.38,0.10
4,Bosnia and Herzegovina,0.06,0.23,0.51,0.17,0.03


Now to make this dataframe long form to be compatible to Altair, we want to "melt" this dataframe so that all ratings (1 to 5) belong to one column, and the corresponding percentage values to another column. To do this, we use Pandas's pd.melt

In [16]:
# create a new dataframe
                             # specify dataframe to melt     # specify which column to keep as a guide
rate_percentage_melt = pd.melt(rate_percentage.reset_index(), id_vars=['index'],
                               # specify which columns to melt
                               # note: column names are normally strings, but in this case the my dataframe
                               # decided that these column names are float instead of string of float
                                value_vars = [1.0, 2.0, 3.0, 4.0, 5.0],
                               # name the column of the rating    # name the column to the value of rating
                                var_name='Rating', value_name='Percentage')

In [17]:
# take a look
rate_percentage_melt

,index,Rating,Percentage
0,Algeria,1.0,0.04
1,Argentina,1.0,0.01
2,Australia,1.0,0.02
3,Belgium,1.0,0.01
4,Bosnia and Herzegovina,1.0,0.06
...,...,...,...
195,Turkey,5.0,0.13
196,Cuba,5.0,0.11
197,Ethiopia,5.0,0.11
198,Vietnam,5.0,0.15


I now proceed to creating a heatmap using the above dataframe

In [18]:
# create a heatmap
chart = alt.Chart(rate_percentage_melt).mark_rect().encode(
    y=alt.Y('index:O',sort='ascending', # sort country name alphabetically
           axis=alt.Axis(labels=True, ticks=False, title=None)),
    x=alt.X('Rating:O', axis=alt.Axis(title=None, ticks=False)),
    color=alt.Color('Percentage:Q', scale=alt.Scale(type='linear', 
                                                    range=['#fde0dd', '#fcc5c0', '#fa9fb5', '#f768a1', 
                                                           '#dd3497', '#ae017e', '#7a0177', '#c51b8a']))
)

In [19]:
# create text that are the percentage to go into each rectange in the heatmap
text = alt.Chart(rate_percentage_melt).mark_text(baseline='middle').encode(
    y=alt.Y('index:O', sort='ascending'), # sort country name alphabetically
    x=alt.X('Rating:O'),
    text='Percentage:Q'
)

In [20]:
# add these two graph together
heat_map = (chart + text).properties(
    width=300,
    height=1000,
    title={'text': ["America's Favorite Cuisine"],
           'subtitle': ['Cuisine Favorability Ratings by',
                        'The Total Number of Respondents Who Rated that Cuisine']}

).configure_view(
    strokeWidth=0,
    
).configure_scale(
    bandPaddingInner=0.05

).configure_axis(domain=False, 
                 labelPadding=10,
                 
).configure_title(
    fontSize=20,
    fontWeight='bold',
    color='#49006a',
    subtitlePadding=7
    
)

In [21]:
heat_map

alt.LayerChart(...)